In [1]:
from music21 import converter, instrument, note, chord, stream, corpus, environment, graph
import music21
import glob

print("Starting to read files...")
streams = []
notes = []
for file in glob.glob("midi_classic/*.mid"):
    print(file)
    midi = converter.parse(file)
    streams.append(midi)
    print("file parsed")
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
print("Files read")

Starting to read files...
midi_classic\appass_1.mid
file parsed
midi_classic\appass_2.mid
file parsed
midi_classic\appass_3.mid
file parsed
midi_classic\beethoven_hammerklavier_1.mid
file parsed
midi_classic\beethoven_hammerklavier_2.mid
file parsed
midi_classic\beethoven_hammerklavier_3.mid
file parsed
midi_classic\beethoven_hammerklavier_4.mid
file parsed
midi_classic\beethoven_les_adieux_1.mid
file parsed
midi_classic\beethoven_les_adieux_2.mid
file parsed
midi_classic\beethoven_les_adieux_3.mid
file parsed
midi_classic\beethoven_opus10_1.mid
file parsed
midi_classic\beethoven_opus10_2.mid
file parsed
midi_classic\beethoven_opus10_3.mid
file parsed
midi_classic\beethoven_opus22_1.mid
file parsed
midi_classic\beethoven_opus22_2.mid
file parsed
midi_classic\beethoven_opus22_3.mid
file parsed
midi_classic\beethoven_opus22_4.mid
file parsed
midi_classic\beethoven_opus90_1.mid
file parsed
midi_classic\beethoven_opus90_2.mid
file parsed
midi_classic\elise.mid
file parsed
midi_classic\mond

In [2]:
def from_graph_to_pianoroll(graph, window_size):
    data_matrix = []
    for item in graph.data:
        if item[0] != '':
            for subitem in item[1]:
                data_matrix.append((item[0], subitem[0], subitem[1]))
    data_matrix = np.array(data_matrix)
    values = []
    for i in range(len(data_matrix)):
        values.append(data_matrix[i][1])
    values = np.array(values)
    values = values.astype(np.float)
    last_bar = max(values)

    sr = 1.0/window_size
    last_bar = last_bar.astype(np.float)
    piano_roll = np.zeros((128, int(np.ceil(last_bar * sr)))) 
    for event in data_matrix:
        pitch = event[0]
        pitch = pitch.replace("\\", "")
        pitch = pitch.replace("$", "")
        pitch = pitch.replace('♯','#')
        pitch = pitch.replace('♭','b')
        event1 = event[1].astype(np.float)
        event2 = event[2].astype(np.float)
        start_idx = int(event1 * sr)
        end_idx = start_idx + int(event2 * sr) + 1
        piano_roll[music21.pitch.Pitch(pitch).midi, start_idx:end_idx] = 1

    return piano_roll

def pianoRollToVertical(pianoRoll):
    w = len(pianoRoll)
    h = len(pianoRoll[0])
    aux = [[0 for x in range(w)] for y in range(h)]
    for x in range(len(pianoRoll)):
        for y in range(len(pianoRoll[x])):
            aux[y][x] = pianoRoll[x][y]

    return aux

def streamToPianoRoll(stream):
    g = graph.plot.HorizontalBarPitchSpaceOffset(stream)
    g.extractData()
    pianoRoll = from_graph_to_pianoroll(g, 1)
    return pianoRoll

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline

n_vocab = len(set(notes))
sequence_length = 10
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(sequence_out)
    
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length))
# normalize input
network_input = network_input / float(n_vocab)


In [4]:
X_train = network_input[0: int(len(network_input) * 0.8)]
X_test = network_input[int(len(network_input) * 0.8): len(network_input)]
Y_train = network_output[0: int(len(network_output) * 0.8)]
Y_test = network_output[int(len(network_output) * 0.8): len(network_output)]

In [5]:
# Models we will use
logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1)
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])

# #############################################################################
# Training


rbm.learning_rate = 0.05
rbm.n_iter = 100
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 1024

# for small values it will underfit the model, for large values it wil overfit the model
# (problem: how big is too big?)
# (solution: generaly you use C as a value close to the magnetute of the training data,
#  in this case I want to overfit so the patterns are learnt since the prediction output
#  will have less diversity than the real output)
logistic.C = 16000000

# Training RBM-Logistic Pipeline
rbm_features_classifier.fit(X_train, Y_train)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -19.41, time = 9.02s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -8.89, time = 19.42s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -5.71, time = 19.37s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -5.24, time = 19.30s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -4.20, time = 19.15s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -3.92, time = 18.96s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -3.40, time = 19.08s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -3.54, time = 18.97s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -3.42, time = 18.91s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -3.32, time = 18.97s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -3.20, time = 18.74s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -3.15, time = 18.67s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -2.86, time = 18.62s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -3.00, time = 18.74s
[BernoulliRBM] Iteration 15, 

Pipeline(steps=[('rbm',
                 BernoulliRBM(learning_rate=0.05, n_components=1024, n_iter=100,
                              random_state=0, verbose=True)),
                ('logistic',
                 LogisticRegression(C=16000000, solver='newton-cg', tol=1))])

In [6]:
# Evaluation
print("Starting prediction 1")
Y_pred2 = rbm_features_classifier.predict(X_test)
print("Finished prediction 1")

Starting prediction 1
Finished prediction 1


In [7]:
# Avoids long repetitions
output = Y_pred2[0:5]
for i in Y_pred2:
    if i == output[len(output) - 4] and i == output[len(output) - 3] and i == output[len(output) - 2] and i == output[len(output) - 1] and i == output[len(output) - 5]:
        continue
    output = np.append(output,i)

In [8]:
for  i in range(0,len(output)-500,500):
    output_notes = []
    offset = 0
    # create note and chord objects based on the values generated by the model
    for pattern in output[i:i+500]:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='RBM_Sklearn_'+ str(int(i/500)) + '.mid')